In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from gurobipy import *
import pandas as pd
import numpy as np

### Model the problem 

Given a set of characters and a set of keyslots we have binary decision variables $x_{cs}$ which are 1 if a character c is assigned to a slot s.
Any combination is possible. 

In addition to the to-be-assigned characters, we have fixed letters. Those are important for the cost functions. For example we care about how long it takes to move from one of the fixed letters / keys to one of the keyslots.

The objectives are computed as follows:
<ol>
<li> Performance:
$$
P = \sum_{c=1}^n\sum_{s=1}^m \sum_{l=1}^{27} (p_{cl} t_{sl} + p_{lc} t_{ls}) x_{cs} 
$$
<li> Association:
$$A = \sum_{c1=1}^n\sum_{c2=1}^n\sum_{s1=1}^m\sum_{s2=1}^m (p_{c1} + p_{c2}) s_{c1c2} d_{s1s2} x_{c1s1}x_{c2s2}
 + \sum_{c=1}^n\sum_{s=1}^m\sum_{l=1}^{27} (p_c + p_l) s_{cl} d_{sl} x_{cs}
$$
<li> Familiarity:
$$
F = \sum_{c=1}^n\sum_{s=1}^m p_{c} d_{s\mathcal{A}(c)} x_{cs}
$$
<li> Ergonomics:
$$
E = \sum_{c=1}^n\sum_{s=1}^m\sum_{l=1}^{27} (p_{cl} e_{sl} + p_{lc} e_{ls}) x_{cs} 
$$

</ol>

In [7]:
def solve_the_keyboard_Problem(azerty,\
                               characters,\
                               keyslots,\
                               letters,\
                               p_single, p_bigram,\
                               performance,\
                               similarity_c_c, similarity_c_l,\
                               distance_level_0, distance_level_1,\
                               ergonomics):
    #Test some stuff in advance to avoid infeasbility:
    if len(characters) > len(keyslots):
        print "Error: more characters than keyslots"
        return
    
    m = Model("keyboard_layout")
    print "creataed model"
    #add decision variables
    x = {}
    for c in characters:
        for s in keyslots:
            x[c,s] = m.addVar(vtype=GRB.BINARY, name=c+" to "+str(s))
    print "created variables"        
    m.update()
    
    P = quicksum(
            ((p_bigram[(c,l)]*performance[(s,azerty[l])]) + (p_bigram[(l,c)]*performance[(azerty[l],s)]))*x[c,s]\
                for c in characters for s in keyslots for l in letters
        )
        
    print "performance"
    
    A = quicksum(
            (p_single[c1] + p_single[c2])*similarity_c_c[(c1,c2)]*distance_level_0[(s1,s2)]\
                 for s1 in keyslots for s2 in keyslots for (c1,c2) in similarity_c_c) +\
        quicksum(
            (p_single[c] + p_single[l])*similarity_c_l[(c,l)]*distance_level_0[s,azerty[l]] \
                 for s in keyslots for (c,l) in similarity_c_l) 
    print "association"   
        
    F = quicksum(
            #if that character was previously not on azerty, distance is 0.
            p_single[c] * distance_level_1.get((s, azerty.get(c,"NaN")),0) \
                 for c in characters for s in keyslots
        )
    print "familiarity"
        
    E = quicksum(
            ((p_bigram[(c,l)]*ergonomics[(s,azerty[l])]) + (p_bigram[(l,c)]*ergonomics[(azerty[l],s)]))*x[c,s]\
                for c in characters for s in keyslots for l in letters
        )
    print "ergonomics"
        
    # Set objective
    m.setObjective(P+A+F+E, GRB.MINIMIZE)
    m.update()
    print "set objective"
    
    #add the constraints. One for each character, one for each keyslot
    for c in characters:
        m.addConstr(quicksum(x[c,s] for s in keyslots) == 1, "characters must be mapped")
        
    for s in keyslots:
        m.addConstr(quicksum(x[c,s] for c in characters) <= 1, "keyslots cann be asigned at most once")
    
    print "set constraints"
    
    m.update()
    print "optimizing..."
    m.optimize()
    
    print "done"
    #Print the solution
    for v in m.getVars():
        if v.x ==1:
            print('%s' % (v.varName))

    print('Obj: %g' % m.objVal)
    
    #return the model with the fixed solution
    return m.fixed()

### Create the input values

#### Some helper functions for reading in stuff

In [8]:
def read_similarity_matrix(path, characters):
    """
        Reads the given matrix into a dictionary of the form (c1,c2)->similarity
        Filters out all characters that are not in the given character list
        Already normalized
    """    
    df = pd.read_excel(path, encoding="utf8")

    index = df.index
    columns = df.columns
    dictionary = {}
    for i in range(0,len(df)): #row index
        for j in range(0, len(df.columns)): #column index
            row = index[i]
            col = columns[j]
            if row in characters and col in characters:
                if not pd.isnull(df.iloc[i,j]):
                    dictionary[(row,col)] = df.iloc[i,j]
    return dictionary

def read_distance_matrix(path, level_cost, recompute=0):
    """
        reads the distance between the keys from the excel file. If the file is empty it creates the distance matrix
        and writes it to the file. The matrix does not include any level distance
        Distance is defined as the sum of the vertical and horizontal distance. 
        Normalizes the distance
        level_cost: dict
            additional cost for each lower level, e.g. level_cost = {'':0, 'Shift':1, 'Alt':2, 'Alt_Shift':3}, 
            is computed in relation to each other, 
            that is cost is 1 if one character on Shift, the other on Alt level. Cost is 2 if one character on single, other
            on Alt level
            
            
    """
    row_numbers = {u"A":0, u"B":1, u"C":2, u"D":3, u"E":4}
    df = pd.read_excel(path)
    index = df.index
    columns = df.columns
    dictionary = {}
    for i in range(0,len(index)): #row index
        for j in range(0, len(columns)): #column index
            slot1 = index[i]
            slot2 = columns[j]            
            if recompute or pd.isnull(df.iloc[i,j]):                
                #if there is no value yet, compute it based on the names:
                row_distance = np.abs(row_numbers[slot1[0]] - row_numbers[slot2[0]])
                column_distance = np.abs(int(slot1[1:3]) - int(slot2[1:3]))                
                #Special case: shift
                if slot1[0:3] == "A03":
                    if int(slot2[1:3])>3:
                        column_distance = max(0,column_distance-4)
                if slot2[0:3] == "A03":
                    if int(slot1[1:3])>3:
                        column_distance = max(0,column_distance-4)
                df.iloc[i,j] = row_distance+column_distance
                
            level_distance = np.abs(level_cost[slot1[4:]] - level_cost[slot2[4:]])
            #add level cost and normalize
            dictionary[(slot1,slot2)] = (df.iloc[i,j]+level_distance) / (df.max().max() + np.max(level_cost.values()))
    df.to_excel(path)                                  
    return dictionary
    

#### Letters:
We only consider the transitions to the lower case letters. To-be-verified assumption: frequency of capital letter-special character pairs is too low.

#### Similarity: 
similarity matrix actually consists of two matrices, one quantifying the similarity between two to-be-assigned characters, one quantifying the similarity between a character and a fixed letter.

In [9]:
print "read in: characters, keyslots and letters"
with open('input\\characters.txt') as f:
    characters = f.read().splitlines()
with open('input\\letters.txt') as f:
    letters = f.read().splitlines()
with open('input\\variable_slots.txt') as f:    
    keyslots = f.read().splitlines()

print "read in: azerty letters"
azerty = pd.read_csv("input\\azerty.csv", index_col=1).to_dict()["keyslot"]

#Similarity values (c,c)->s, (c,l)->s
print "read in: similarity values"
similarity_c_c = read_similarity_matrix('input\\similarity_c_c.xlsx', characters)
similarity_c_l = read_similarity_matrix('input\\similarity_c_l.xlsx', characters)

#Distance values (key,key)->d
print "read in: distance values"
level_cost = {u'':0, u'Shift':0, u'Alt':0, u'Alt_Shift':0}
distance_level_0 = read_distance_matrix("input\\distance.xlsx", level_cost)
level_cost = {u'':0, u'Shift':1, u'Alt':2, u'Alt_Shift':3}
distance_level_1 = read_distance_matrix("input\\distance.xlsx", level_cost)

#Frequency distributions c/l -> p, (c/l, c/l) -> p
#TODO
all_chars = letters+characters
p_single = {character: float(1)/len(all_chars) for character in all_chars}
p_bigram = {(c1,c2): float(1)/(len(all_chars) * len(all_chars) )for c1 in all_chars for c2 in all_chars}

#Ergonomics (c, l)-> e
#TODO
ergonomics = {}
for s in keyslots:
    for l in letters:
        ergonomics[(s,azerty[l])]= 0.2 
        ergonomics[(azerty[l],s)]= 0.2 

#Performance: (key, letter)->t
#TODO
performance = {}
for s in keyslots:
    for l in letters:
        performance[(s,azerty[l])]= 0.2 
        performance[(azerty[l],s)]= 0.2 

print "Done reading input values."


read in: characters, keyslots and letters
read in: azerty letters
read in: similarity values
read in: distance values
Done reading input values.


C:\Users\feita1\Python_Anaconda\lib\site-packages\ipykernel\__main__.py:16: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


### Optimize

In [ ]:
m = solve_the_keyboard_Problem(azerty,\
                               characters,\
                               keyslots,\
                               letters,\
                               p_single, p_bigram,\
                               performance,\
                               similarity_c_c, similarity_c_l,\
                               distance_level_0, distance_level_1,\
                               ergonomics)

In [11]:
F = quicksum(
            #if that character was previously not on azerty, distance is 0.
            p_single[c] * distance_level_1.get((s, azerty.get(c,"NaN")),0) \
                 for c in characters for s in keyslots
        )

In [ ]:
azerty.get[c,"NaN"]

# Notes:
how to encode properly to utf-8 such that you can write it to file:

In [28]:
foo = u'Δ, Й, ק, ‎ م, ๗, あ, 叶, 葉, and 말.'
f = open('test', 'w')
f.write(foo.encode('utf8'))
f.close()